In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from tsm.data_utils import get_data_sample, time_processing, df_to_x_y, train_dev_test_split

/Users/Ric/_macos_programs/environments/python/x_tsm/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
import numpy as np

In [5]:
from tsm.model_baselines import dummy_regressor_accuracy
from tsm.eval_metrics  import root_mean_squared_log_error

In [77]:
from tsm.data_utils import ordinal2wave

## Load and Prepare Data for Experiments

In [6]:
train_data = pd.read_pickle('kaggle/input/ashrae-energy-prediction/train.pkl')

### Full Training Data

In [7]:
trdf, dvdf, tsdf = train_dev_test_split(train_data)

In [8]:
x_idx = [0, 1] + [x for x in range(4, len(list(train_data)))]
y_idx = 3

In [9]:
tr_x, tr_y = df_to_x_y(trdf, x_idx, y_idx)
dv_x, dv_y = df_to_x_y(dvdf, x_idx, y_idx)
ts_x, ts_y = df_to_x_y(tsdf, x_idx, y_idx)

In [10]:
# Dummy regressor
dummy = dummy_regressor_accuracy(tr_x, tr_y, root_mean_squared_log_error)

DummyRegressor accuracy: 4.191078645829351


In [16]:
# Smarter Dummy: takes the mean by meter type
def smart_dummy_predict(tr_df, tr_x, meter_idx_in_df: int = 1):
    mean_by_type = tr_df.groupby('meter')['meter_reading'].mean().to_dict()
    return np.array([mean_by_type[r] for r in tr_x[:, meter_idx_in_df]])

In [17]:
# Smart dummy training accuracy
root_mean_squared_log_error(smart_dummy_predict(trdf, tr_x), tr_y)

3.0861123708683027

In [18]:
# Smart dummy test accuracy
root_mean_squared_log_error(smart_dummy_predict(trdf, ts_x), ts_y)

2.907413743289485

## Let me experiment with a single site before scaling

In [27]:
train_data.building_id.value_counts()

/Users/Ric/_macos_programs/environments/python/x_tsm/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


1298    35136
1249    35136
1301    35128
1241    35116
1296    35115
        ...  
783      2657
420      2327
53       1685
604      1012
403       479
Name: building_id, Length: 1449, dtype: int64

In [121]:
sample_site = train_data[train_data.building_id == 1241]

In [122]:
len(sample_site)

35116

In [123]:
ordinal2wave('dt_m', sample_site)
ordinal2wave('dt_m', sample_site)
ordinal2wave('dt_w', sample_site)
ordinal2wave('dt_d', sample_site)
ordinal2wave('dt_hour', sample_site)
ordinal2wave('dt_day_week', sample_site)
ordinal2wave('dt_day_month', sample_site)
ordinal2wave('dt_week_month', sample_site)

/Users/Ric/_personal/x_tsm/tsm/data_utils.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col_name}_sin'] = np.sin(2 * np.pi * df[col_name] / df[col_name].max())
/Users/Ric/_personal/x_tsm/tsm/data_utils.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col_name}_cos'] = np.cos(2 * np.pi * df[col_name] / df[col_name].max())


In [124]:
tr_smp, ts_smp = train_dev_test_split(sample_site, train_pct=0.75, dev_pct=0)

In [125]:
x_idx = [0, 1] + [x for x in range(4, len(list(sample_site)))]
y_idx = 3

In [126]:
tr_smpx, tr_smpy = df_to_x_y(tr_smp, x_idx, y_idx)
ts_smpx, ts_smpy = df_to_x_y(ts_smp, x_idx, y_idx)

In [51]:
# Smart dummy training accuracy
root_mean_squared_log_error(smart_dummy_predict(tr_smp, tr_smpx), tr_smpy)

2.854176101352186

In [52]:
# Smart dummy training accuracy
root_mean_squared_log_error(smart_dummy_predict(tr_smp, ts_smpx), ts_smpy)

3.832249108691976

In [127]:
# Let's try a Random Forest regression
from sklearn.ensemble import RandomForestRegressor
rfrg = RandomForestRegressor(n_jobs=-1, max_depth=5, n_estimators=39)

In [128]:
rfrg.fit(tr_smpx, tr_smpy)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=39, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [129]:
root_mean_squared_log_error(rfrg.predict(tr_smpx), tr_smpy)

0.7046630115659629

In [130]:
root_mean_squared_log_error(rfrg.predict(ts_smpx), ts_smpy)

1.7567391977444955

In [134]:
pd.set_option('display.max_columns', 200)
sample_site.describe()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,dt_m,dt_w,dt_d,dt_hour,dt_day_week,dt_day_month,dt_week_month,dt_m_sin,dt_m_cos,dt_w_sin,dt_w_cos,dt_d_sin,dt_d_cos,dt_hour_sin,dt_hour_cos,dt_day_week_sin,dt_day_week_cos,dt_day_month_sin,dt_day_month_cos,dt_week_month_sin,dt_week_month_cos
count,35116.0,35116.000000,35116.000000,35116.0,35116.0,35116.0,35116.0,35116.0,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,3.511600e+04,3.511600e+04,3.511600e+04,35116.000000,3.511600e+04,35116.000000,35116.000000,35116.000000,35116.000000,35116.000000,3.511600e+04,35116.000000,3.511600e+04,35116.000000
mean,1241.0,1.499231,1161.941650,14.0,3.0,194188.0,9999.0,99.0,13.334033,37.741001,5.857646,5.063959,1136.098145,215.140677,3.403981,6.514466,26.651669,183.525829,11.499801,3.006863,15.757774,2.705092,-3.258210e-03,-5.074021e-04,-9.512725e-05,-0.013280,-3.909839e-04,0.000399,0.000049,0.041569,-0.004439,0.141787,4.050461e-04,-0.015820,-6.695355e-05,-0.147164
std,0.0,1.117795,1165.212280,0.0,0.0,0.0,0.0,0.0,10.572361,47.738782,10.970021,65.778553,1030.093384,189.904433,5.968377,3.452085,15.102369,105.680366,6.921934,1.997702,8.814005,1.272251,7.062429e-01,7.079820e-01,7.118772e-01,0.702199,7.071145e-01,0.707119,0.692215,0.720513,0.655980,0.741347,7.129733e-01,0.701033,7.575730e-01,0.635967
min,1241.0,0.000000,0.000000,14.0,3.0,194188.0,9999.0,99.0,-15.600000,0.000000,-25.600000,-1.000000,989.599976,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,-1.000000e+00,-1.000000e+00,-9.995608e-01,-0.998244,-9.999632e-01,-1.000000,-0.997669,-0.990686,-0.866025,-1.000000,-9.987165e-01,-0.994869,-9.510565e-01,-0.809017
25%,1241.0,0.000000,176.592995,14.0,3.0,194188.0,9999.0,99.0,5.000000,0.000000,-2.800000,0.000000,1011.799988,70.000000,1.500000,4.000000,14.000000,92.000000,6.000000,1.000000,8.000000,2.000000,-8.660254e-01,-8.660254e-01,-6.965510e-01,-0.717507,-7.101350e-01,-0.704066,-0.631088,-0.775711,-0.866025,-0.500000,-7.247928e-01,-0.758758,-5.877853e-01,-0.809017
50%,1241.0,1.000000,850.369995,14.0,3.0,194188.0,9999.0,99.0,13.300000,0.000000,6.100000,0.000000,1016.700012,220.000000,2.600000,7.000000,27.000000,184.000000,11.000000,3.000000,16.000000,3.000000,-2.449294e-16,6.123234e-17,-2.449294e-16,0.029633,-2.449294e-16,0.008583,0.000000,-0.068242,0.000000,0.500000,-2.449294e-16,-0.050649,-2.449294e-16,0.309017
75%,1241.0,2.000000,1858.587524,14.0,3.0,194188.0,9999.0,99.0,21.700001,99.000000,15.000000,0.000000,1021.200012,300.000000,4.100000,10.000000,40.000000,275.000000,17.000000,5.000000,23.000000,4.000000,5.000000e-01,8.660254e-01,6.965510e-01,0.674983,7.101350e-01,0.704066,0.656025,0.682553,0.866025,1.000000,7.247928e-01,0.688967,5.877853e-01,0.309017
max,1241.0,3.000000,6148.330078,14.0,3.0,194188.0,9999.0,99.0,99.000000,99.000000,99.000000,999.000000,9999.000000,999.000000,99.000000,12.000000,53.000000,366.000000,23.000000,6.000000,31.000000,5.000000,1.000000e+00,1.000000e+00,9.995608e-01,1.000000,9.999632e-01,1.000000,0.997669,1.000000,0.866025,1.000000,9.987165e-01,1.000000,9.510565e-01,1.000000


## Time Series Modelling

### Let's start using no feature other than meter reading and meter type

In [146]:
from datetime import timedelta

In [139]:
sample_site_ts = sample_site[['meter', 'meter_reading', 'timestamp']]

In [160]:
from tsm.data_utils import add_k_past_lags

/Users/Ric/_macos_programs/environments/python/x_tsm/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [195]:
sample_site_ts = sample_site_ts.progress_apply(lambda r: add_k_past_lags(r, sample_site_ts, 'timestamp', 'meter_reading', 'meter'), axis=1)

In [198]:
sample_site_ts.dropna(inplace=True)

In [199]:
sample_site_ts

,meter,meter_reading,timestamp,meter_reading_t-1,meter_reading_t-2,meter_reading_t-3,meter_reading_t-4,meter_reading_t-5,meter_reading_t-6,meter_reading_t-7,meter_reading_t-8,meter_reading_t-9,meter_reading_t-10
24785,0,174.171997,2016-01-01 10:00:00,234.179993,294.156006,354.152008,294.121002,294.101990,294.089996,234.147995,234.164001,234.151993,234.160004
24786,1,1050.310059,2016-01-01 10:00:00,2249.919922,2080.939941,1995.560059,2011.180054,1988.310059,1785.680054,1149.339966,1122.380005,1140.790039,1144.390015
24787,2,1241.439941,2016-01-01 10:00:00,2398.879883,2548.489990,2207.290039,2661.340088,2249.280029,2225.659912,2186.290039,1805.719971,1771.609985,2251.909912
24788,3,1689.939941,2016-01-01 10:00:00,1331.800049,1466.300049,2528.439941,2692.550049,2547.929932,2249.989990,1786.969971,1656.900024,1638.579956,1922.689941
27084,0,234.171997,2016-01-01 11:00:00,294.156006,354.152008,294.121002,294.101990,294.089996,234.147995,234.164001,234.151993,234.160004,174.171997
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20213257,3,0.760000,2016-12-31 22:00:00,1.090000,1.110000,1.110000,1.120000,1.110000,1.080000,1.030000,0.950000,0.870000,0.810000
20215619,0,260.000000,2016-12-31 23:00:00,264.000000,273.000000,260.000000,276.000000,282.000000,266.000000,262.000000,252.000000,252.000000,257.000000
20215620,1,617.379028,2016-12-31 23:00:00,563.203003,551.807007,559.666016,616.874023,676.291992,692.562988,691.482971,714.997009,702.973999,662.281982
20215621,2,1965.829956,2016-12-31 23:00:00,2947.429932,3034.040039,2973.669922,2753.209961,1703.369995,2349.020020,1918.579956,1942.199951,1989.449951,1860.839966


In [241]:
tr_smpts, ts_smpts = train_dev_test_split(sample_site_ts, train_pct=0.75, dev_pct=0)

In [242]:
tsx_idx = [0] + [x for x in range(3, len(list(sample_site_ts)))]
tsy_idx = 1

In [243]:
tr_smpx, tr_smpy = df_to_x_y(tr_smpts, tsx_idx, tsy_idx)
ts_smpx, ts_smpy = df_to_x_y(ts_smpts, tsx_idx, tsy_idx)

In [254]:
rfrg = RandomForestRegressor(n_jobs=-1, max_depth=25, n_estimators=39)
rfrg.fit(tr_smpx, tr_smpy)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=39, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [255]:
root_mean_squared_log_error(rfrg.predict(tr_smpx), tr_smpy)

0.4863791492424107

In [256]:
root_mean_squared_log_error(rfrg.predict(ts_smpx), ts_smpy)

0.4455103092395611

In [222]:
# 10 past lags + features?
sample_ts_with_feats = sample_site_ts.merge(sample_site)

In [223]:
tr_smptsf, ts_smptsf = train_dev_test_split(sample_ts_with_feats, train_pct=0.75, dev_pct=0)

In [224]:
sample_ts_with_feats.head()

,meter,meter_reading,timestamp,meter_reading_t-1,meter_reading_t-2,meter_reading_t-3,meter_reading_t-4,meter_reading_t-5,meter_reading_t-6,meter_reading_t-7,meter_reading_t-8,meter_reading_t-9,meter_reading_t-10,building_id,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,dt_m,dt_w,dt_d,dt_hour,dt_day_week,dt_day_month,dt_week_month,dt_m_sin,dt_m_cos,dt_w_sin,dt_w_cos,dt_d_sin,dt_d_cos,dt_hour_sin,dt_hour_cos,dt_day_week_sin,dt_day_week_cos,dt_day_month_sin,dt_day_month_cos,dt_week_month_sin,dt_week_month_cos
0,0,174.171997,2016-01-01 10:00:00,234.179993,294.156006,354.152008,294.121002,294.101990,294.089996,234.147995,234.164001,234.151993,234.160004,1241,14,3,194188,9999,99,3.3,0,-3.3,0,1018.000000,280,2.6,1,53,1,10,4,1,1,0.5,0.866025,-2.449294e-16,1.0,0.017166,0.999853,0.398401,-0.917211,-0.866025,-0.5,0.201299,0.97953,0.951057,0.309017
1,1,1050.310059,2016-01-01 10:00:00,2249.919922,2080.939941,1995.560059,2011.180054,1988.310059,1785.680054,1149.339966,1122.380005,1140.790039,1144.390015,1241,14,3,194188,9999,99,3.3,0,-3.3,0,1018.000000,280,2.6,1,53,1,10,4,1,1,0.5,0.866025,-2.449294e-16,1.0,0.017166,0.999853,0.398401,-0.917211,-0.866025,-0.5,0.201299,0.97953,0.951057,0.309017
2,2,1241.439941,2016-01-01 10:00:00,2398.879883,2548.489990,2207.290039,2661.340088,2249.280029,2225.659912,2186.290039,1805.719971,1771.609985,2251.909912,1241,14,3,194188,9999,99,3.3,0,-3.3,0,1018.000000,280,2.6,1,53,1,10,4,1,1,0.5,0.866025,-2.449294e-16,1.0,0.017166,0.999853,0.398401,-0.917211,-0.866025,-0.5,0.201299,0.97953,0.951057,0.309017
3,3,1689.939941,2016-01-01 10:00:00,1331.800049,1466.300049,2528.439941,2692.550049,2547.929932,2249.989990,1786.969971,1656.900024,1638.579956,1922.689941,1241,14,3,194188,9999,99,3.3,0,-3.3,0,1018.000000,280,2.6,1,53,1,10,4,1,1,0.5,0.866025,-2.449294e-16,1.0,0.017166,0.999853,0.398401,-0.917211,-0.866025,-0.5,0.201299,0.97953,0.951057,0.309017
4,0,234.171997,2016-01-01 11:00:00,294.156006,354.152008,294.121002,294.101990,294.089996,234.147995,234.164001,234.151993,234.160004,174.171997,1241,14,3,194188,9999,99,3.3,0,-3.3,0,1017.900024,280,2.6,1,53,1,11,4,1,1,0.5,0.866025,-2.449294e-16,1.0,0.017166,0.999853,0.136167,-0.990686,-0.866025,-0.5,0.201299,0.97953,0.951057,0.309017


In [225]:
tsfx_idx = [0] + [x for x in range(3, len(list(sample_ts_with_feats)))]
tsfy_idx = 1

In [226]:
tr_smfpx, tr_smfpy = df_to_x_y(tr_smptsf, tsfx_idx, tsfy_idx)
ts_smfpx, ts_smfpy = df_to_x_y(ts_smptsf, tsfx_idx, tsfy_idx)

In [251]:
rfrg = RandomForestRegressor(n_jobs=-1, max_depth=5, n_estimators=39)
rfrg.fit(tr_smfpx, tr_smfpy)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=39, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [252]:
root_mean_squared_log_error(rfrg.predict(tr_smfpx), tr_smfpy)

0.6806889120941725

In [253]:
root_mean_squared_log_error(rfrg.predict(ts_smfpx), ts_smfpy)

2.0278930312016406

In [257]:
tr_smptsf.shape

(26292, 47)

In [259]:
# Save processed features before leaving
sample_ts_with_feats.to_pickle('kaggle/input/ashrae-energy-prediction/store_1241_10lags_train.pkl')